In [ ]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [19]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib

importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [20]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [21]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain(domain='grammar')
task_type = await ui.choose_task_type(task_type='text_summary')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [22]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a short text that the reader has to summarize in 5 to 6 sentences. only give a text of at least 250 words and a task description asking the student to write a summary, do not write a summary yourself.. Generate it for the area of vocabulary teaching, targeting on Colloquial Language.
Adjust the task dificulty to language level B2 (Can understand the main ideas of complex text on both concrete and abstract topics, including technical discussions in his/her field of specialisation. Can interact with a degree of fluency and spontaneity that makes regular interaction with native speakers quite possible without strain for either party. Can produce clear, detailed text on a wide range of subjects and explain a viewpoint on a topical issue giving the advantages and disadvantages of various options).


In [23]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Text:

In everyday conversation, people often use colloquial language to express themselves and communicate effectively. Colloquial language refers to informal words, phrases, and expressions that are commonly used in casual settings. It is the language used among friends, family members, and colleagues in relaxed and informal situations.

Colloquial language varies based on region, culture, and social group. It includes slang words, idiomatic expressions, and informal vocabulary. For example, in American English, people might use the word "cool" to mean something is great or impressive, whereas in British English, people might use the word "brilliant" for the same purpose. Moreover, idiomatic expressions like "it's raining cats and dogs" are commonly used in colloquial language but may not make sense to non-native speakers.

Colloquial language serves several purposes. Firstly, it fosters a sense of belonging and camaraderie among individuals who share similar backgrounds and experien

In [ ]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="b)")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


In [ ]:
######################
# This is what it will actually looks like
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

In [ ]:
# While the parsing is not implemented, this function will not work
#new_profile = await evaluator.update_learner_profile(training_goals, profile)
#print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

In [ ]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
# await db.save_profile(new_profile)